In [1]:
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Test Installation

In [2]:
dl = tf.constant('Hello world, Deep Learning and Tensorflow!')
val1 = tf.constant(2)
val2 = tf.constant(3)
mul_val = tf.multiply(val1, val2)

with tf.Session() as sess:
    print(sess.run(dl))
    print(sess.run(mul_val))

b'Hello world, Deep Learning and Tensorflow!'
6


# Tensor vs. Variable

Tensor is different from variable. Tensor is active data flow between the layers in neural network and it vanishs after sesson finish. Variable can be preserved among different sessions, it can be updated during training, it can be kept storing in memory and it can be stored after learning.

In [3]:
"""
Others tensors to initialize the tf.variables
tf.zeros()
tf.ones()
tf.random_normal()
tf.truncated_normal()
tf.random_uniform()
"""
weights = tf.Variable(tf.random_normal([300,200], stddev=0.5), name="weights", trainable=True)

Requires `tf.initialize_all_variables()` or `tf.initialize_variables(var1, var2, var3, ...)` to initialize tensors in variables.

# Operations in Tensorflow

Operations in tensorflow represent to transform tensor.

| Class | Examples |
|--|--|
| Element-based Calculation | Add, Sub, Mul, Div, Exp, Log, Equal, ... |
| Array Operation | Concat, Slice, Split, Constant, Rank, Shape, Shuffle, ... |
| Matrix Operation | MatMul, MatrixInverse, MatrixDeterminant, ... |
| Status Operation | Variable, Assign, AssignAdd, ... |
| Neural Network Construction | Softmax, Sigmoid, ReLu, Convolutional2D, Maxpool, ... |
| Checkpoint Operation | Save, Restore, ... |
| Queue, Sync Operation | Enqueue, Dequeue, MutexAcquire, MutexRelease, ... |
| Flow Control | Merge, Switch, Enter, Leave, NextIteration, ... |

# Placeholder

Placeholder is the data entry point (tensor) to neural network model.

In [4]:
x = tf.placeholder(tf.float32, name="x", shape=[None, 784])   # None can be any size in minibatch
W = tf.Variable(tf.random_uniform([784,10], minval=-1, maxval=1), name="W")
multiply_val = tf.matmul(x, W)

# Session in Tensorflow

* Tensor(Data/Operations), Variables(Weights) and Graphs(Nerual Network) are executed in session.
* All operations listed above, including initialization, are working in session.

```python
import tensorflow as tf
from read_data import get_minibatch

# simple neural network
x = tf.placeholder(tf.float32, name="x", shape=[None, 784])   # None can be any size in minibatch
W = tf.Variable(tf.random_uniform([784,10], minval=-1, maxval=1), name="W")
b = tf.Variable(tf.zeros([10]), name="biases")
output = tf.matmul(x, W) + b

# initialization
init = tf.initialize_all_variables()

# start the session
with tf.Session() as sess:
    # initialize all variables at the beginning
    sess.run(init)
    
    # use feed_dict to fill the data into placeholder
    # and calculate around the whole graph
    feed_dict_data = {"x": get_minibatch()}
    sess.run(output, feed_dict = feed_dict_data)
```

# Scope in Tensorflow

* Use `tf.get_variable()` instead of `tf.Variable()` to check whether the variable is initialized already, if it is not, tensorflow would initialize the variable as the same with `tf.Variable()`.

* Use `tf.variable_scope()` as `scope` and script `scope.reuse_variables()` to allow reusing shared variables among minibatch training. (For the sake of information security, tensorflow is not allowed to shared variables by default.) 

## Better Variable Construction

In [5]:
def layer(input, weight_shape, bias_shape):
    weight_init = tf.random_uniform_initializer(minval=-1, maxval=1)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", shape=weight_shape, initializer=weight_init)
    b = tf.get_variable("b", shape=bias_shape, initializer=bias_init)
    return tf.matmul(input, W) + b

def network(input):
    """
    A four-layer neural network example 
    """
    
    with tf.variable_scope("layer_1"):
        output_1 = layer(input, [784,100], [100])
    
    with tf.variable_scope("layer_2"):
        output_2 = layer(output_1, [100, 50], [50])
        
    with tf.variable_scope("layer_3"):
        output_3 = layer(output_2, [50, 10], [10])
        
    return output_3

In [6]:
with tf.variable_scope("shared_variables") as scope:
    i_1 = tf.placeholder(tf.float32, [1000, 784], name="i_1")
    network(i_1)
    
    # you should add the script to allow sharing variables
    scope.reuse_variables()
    
    i_2 = tf.placeholder(tf.float32, [1000, 784], name="i_2")
    network(i_2)

# Use CPU / GPU

```text
# use the first CPU
/cpu:0

# use the first gpu
/gpu:0

# use the second gpu
/gpu:1
```

In [7]:
# check supported devices
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7154796531582009912]

## Basic use

In [8]:
with tf.device('/gpu:1'):
    
    # use tf.device() to assign the computing resource
    a = tf.constant([1., 2., 3., 4.], shape=[2,2], name='a')
    b = tf.constant([10., -10.], shape=[2,1], name='b')
    c = tf.matmul(a, b)
    
    # allow_soft_placement: if device is not available to use, we allow tensorflow to change CPU or GPU:0, ...
    # log_device_placement: record which devices we use
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        print(sess.run(c))

[[-10.]
 [-10.]]


## Tower-like fashion way

In [9]:
c = []

for d in ['/gpu:0', '/gpu:1']:
    with tf.device(d):
        a = tf.constant([1., 2., 3., 4.], shape=[2,2], name='a')
        b = tf.constant([10., -10.], shape=[2,1], name='b')
        c.append(tf.matmul(a,b))
        
with tf.device('/cpu:0'):
    sumAll = tf.add_n(c)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print(sess.run(sumAll))

[[-20.]
 [-20.]]


# MNIST Example

## Inference

In [10]:
def old_inference(x):
    init = tf.constant_initializer(value=0)
    W = tf.get_variable(name="W", shape=[784,10], initializer=init)
    b = tf.get_variable(name="b", shape=[10], initializer=init)
    output = tf.nn.softmax(tf.matmul(x, W) + b)
    return output

In [11]:
def layer(input, weight_shape, bias_shape):
    weight_std = (2.0 / weight_shape[0]) ** 0.5
    w_init = tf.random_normal_initializer(stddev=weight_std)
    b_init = tf.constant_initializer(value=0)
    W = tf.get_variable(name="W", shape=weight_shape, initializer=w_init)
    b = tf.get_variable(name="b", shape=bias_shape, initializer=b_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

In [12]:
def inference(x):
    with tf.variable_scope("hidden_1"):
        hidden_1 = layer(x, [784, 256], [256])
        
    with tf.variable_scope("hidden_2"):
        hidden_2 = layer(hidden_1, [256, 256], [256])
    
    with tf.variable_scope("output"):
        output = layer(hidden_2, [256, 10], [10])
        
    return output

## Loss

In [13]:
def old_loss(output, y):
    product = y * tf.log(output)
    # 0: column, 1: row
    cross_entropy = -tf.reduce_sum(product, reduction_indices=1)
    loss = tf.reduce_mean(cross_entropy)
    return loss

In [14]:
def loss(output, y):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y)
    loss = tf.reduce_mean(cross_entropy)
    return loss

## Training

In [15]:
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_opt = optimizer.minimize(cost, global_step=global_step)
    return train_opt

## Evaluate

In [16]:
def evaluate(output, y, name):
    compare = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))
    tf.summary.scalar("eval/{}".format(str(name)), accuracy)
    return accuracy

## Hyperparameters

In [17]:
full_training = True
learning_rate = 1e-2
training_epochs = 1000 if full_training else 1
batch_size = 100 if full_training else 1000
display_step = 1000 if full_training else 100

In [18]:
# uncomment the below line to download the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Flow

In [19]:
with tf.Graph().as_default():
    with tf.variable_scope("mlp_model"):
        x = tf.placeholder("float", [None, 784])   # x is batch input
        y = tf.placeholder("float", [None, 10])   # y is output for 10 classification

        output = inference(x)   # get the inference result
        cost = loss(output=output, y=y)   # get the loss
        global_step = tf.Variable(0, name="global_step", trainable=False)   # training step
        train_opt = training(cost=cost, global_step=global_step)   # training body
        eval_opt_val = evaluate(output=output, y=y, name="valid")   # evaluation result
        eval_opt_test = evaluate(output=output, y=y, name="test")   # evaluation result

        init_var = tf.global_variables_initializer()
        summary_opt = tf.summary.merge_all()   # merge summary in each epoch
        saver = tf.train.Saver()   # for saving checkpoints

        with tf.Session() as sess:
            # write the summary 
            summary_writer = tf.summary.FileWriter("/Users/jiankaiwang/Desktop/devops/tmp/basic_tf/logistic_logs_nn", graph=sess.graph)
            sess.run(init_var)   # initialize all variables

            for epoch in range(training_epochs):
                avg_cost = 0.
                total_batch = int(mnist.train.num_examples / batch_size)

                for idx in range(total_batch):
                    batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)   # get the batch data

                    feed_dict_data = {x: batch_x, y: batch_y}
                    sess.run(train_opt, feed_dict=feed_dict_data)   # feed into data and start training

                    batch_cost = sess.run(cost, feed_dict=feed_dict_data)
                    avg_cost += batch_cost / total_batch   # calculate the average loss

                    if epoch % display_step == 0:
                        # record log

                        feed_dict_val_data = {x: mnist.validation.images, y: mnist.validation.labels}
                        acc = sess.run(eval_opt_val, feed_dict=feed_dict_val_data)   # calculate the accuracy

                        print("Accuracy:", acc, "Vaildation Error:", (1-acc))
                        tf.summary.scalar("validation_accuracy", acc)  

                        summary_str = sess.run(summary_opt, feed_dict=feed_dict_data)
                        summary_writer.add_summary(summary_str, sess.run(global_step))   # write out the summary

                        saver.save(sess, \
                                   "/Users/jiankaiwang/Desktop/devops/tmp/basic_tf/model-checkpoint", \
                                  global_step=global_step)

            print("Training finishing.")

            feed_dict_test_data = {x: mnist.test.images, y: mnist.test.labels}
            acc = sess.run(eval_opt_test, feed_dict=feed_dict_test_data)   # test result
            print("Test Accuracy:",acc)

Accuracy: 0.1198 Vaildation Error: 0.8801999986171722
Accuracy: 0.1252 Vaildation Error: 0.8747999966144562
Accuracy: 0.1322 Vaildation Error: 0.8677999973297119
Accuracy: 0.1386 Vaildation Error: 0.8613999933004379
Accuracy: 0.142 Vaildation Error: 0.8579999953508377
Accuracy: 0.1534 Vaildation Error: 0.8465999960899353
Accuracy: 0.164 Vaildation Error: 0.835999995470047
Accuracy: 0.171 Vaildation Error: 0.8289999961853027
Accuracy: 0.1826 Vaildation Error: 0.8173999935388565
Accuracy: 0.1906 Vaildation Error: 0.8094000071287155
Accuracy: 0.1982 Vaildation Error: 0.8017999976873398
Accuracy: 0.2046 Vaildation Error: 0.7953999936580658
Accuracy: 0.2114 Vaildation Error: 0.7885999977588654
Accuracy: 0.2192 Vaildation Error: 0.780799999833107
Accuracy: 0.221 Vaildation Error: 0.778999999165535
Accuracy: 0.226 Vaildation Error: 0.7740000039339066
Accuracy: 0.2314 Vaildation Error: 0.7686000019311905
Accuracy: 0.2362 Vaildation Error: 0.7637999951839447
Accuracy: 0.241 Vaildation Error: 0.

Accuracy: 0.4238 Vaildation Error: 0.576200008392334
Accuracy: 0.4278 Vaildation Error: 0.5722000002861023
Accuracy: 0.4324 Vaildation Error: 0.5676000118255615
Accuracy: 0.4384 Vaildation Error: 0.561599999666214
Accuracy: 0.4416 Vaildation Error: 0.5584000051021576
Accuracy: 0.4416 Vaildation Error: 0.5584000051021576
Accuracy: 0.4416 Vaildation Error: 0.5584000051021576
Accuracy: 0.4424 Vaildation Error: 0.5575999915599823
Accuracy: 0.4464 Vaildation Error: 0.553600013256073
Accuracy: 0.4508 Vaildation Error: 0.5491999983787537
Accuracy: 0.4534 Vaildation Error: 0.5466000139713287
Accuracy: 0.4514 Vaildation Error: 0.5485999882221222
Accuracy: 0.4548 Vaildation Error: 0.545199990272522
Accuracy: 0.4546 Vaildation Error: 0.5453999936580658
Accuracy: 0.4586 Vaildation Error: 0.5413999855518341
Accuracy: 0.4626 Vaildation Error: 0.5374000072479248
Accuracy: 0.4618 Vaildation Error: 0.5381999909877777
Accuracy: 0.4638 Vaildation Error: 0.5361999869346619
Accuracy: 0.4632 Vaildation Erro

Accuracy: 0.6588 Vaildation Error: 0.34119999408721924
Accuracy: 0.6652 Vaildation Error: 0.33480000495910645
Accuracy: 0.6644 Vaildation Error: 0.33560001850128174
Accuracy: 0.6644 Vaildation Error: 0.33560001850128174
Accuracy: 0.666 Vaildation Error: 0.33399999141693115
Accuracy: 0.6662 Vaildation Error: 0.3338000178337097
Accuracy: 0.6654 Vaildation Error: 0.33459997177124023
Accuracy: 0.6716 Vaildation Error: 0.32840001583099365
Accuracy: 0.6696 Vaildation Error: 0.3303999900817871
Accuracy: 0.6654 Vaildation Error: 0.33459997177124023
Accuracy: 0.6702 Vaildation Error: 0.329800009727478
Accuracy: 0.6724 Vaildation Error: 0.32760000228881836
Accuracy: 0.6696 Vaildation Error: 0.3303999900817871
Accuracy: 0.6748 Vaildation Error: 0.32520002126693726
Accuracy: 0.6768 Vaildation Error: 0.323199987411499
Accuracy: 0.6802 Vaildation Error: 0.3198000192642212
Accuracy: 0.681 Vaildation Error: 0.3190000057220459
Accuracy: 0.6776 Vaildation Error: 0.32239997386932373
Accuracy: 0.6784 Vail

Accuracy: 0.714 Vaildation Error: 0.28600001335144043
Accuracy: 0.7138 Vaildation Error: 0.28619998693466187
Accuracy: 0.7154 Vaildation Error: 0.28460001945495605
Accuracy: 0.7158 Vaildation Error: 0.2842000126838684
Accuracy: 0.7146 Vaildation Error: 0.2853999733924866
Accuracy: 0.7158 Vaildation Error: 0.2842000126838684
Accuracy: 0.716 Vaildation Error: 0.2839999794960022
Accuracy: 0.7148 Vaildation Error: 0.28519999980926514
Accuracy: 0.7162 Vaildation Error: 0.28380000591278076
Accuracy: 0.7164 Vaildation Error: 0.28359997272491455
Accuracy: 0.7166 Vaildation Error: 0.2833999991416931
Accuracy: 0.7162 Vaildation Error: 0.28380000591278076
Accuracy: 0.716 Vaildation Error: 0.2839999794960022
Accuracy: 0.7168 Vaildation Error: 0.2832000255584717
Accuracy: 0.716 Vaildation Error: 0.2839999794960022
Accuracy: 0.7158 Vaildation Error: 0.2842000126838684
Accuracy: 0.7152 Vaildation Error: 0.2847999930381775
Accuracy: 0.714 Vaildation Error: 0.28600001335144043
Accuracy: 0.7146 Vaildati

KeyboardInterrupt: 